In [1]:
# Visualization of the KO+ChIP Gold Standard from:
# Miraldi et al. (2018) "Leveraging chromatin accessibility for transcriptional regulatory network inference in Th17 Cells"

# TO START: In the menu above, choose "Cell" --> "Run All", and network + heatmap will load
# Change "canvas" to "SVG" (drop-down menu in cell below) to enable drag interactions with nodes & labels
# More info about jp_gene_viz and user interface instructions are available on Github: 
# https://github.com/simonsfoundation/jp_gene_viz/blob/master/doc/dNetwork%20widget%20overview.ipynb
# Info specific to the "Multi-network" view:
# https://github.com/simonsfoundation/jp_gene_viz/blob/master/doc/Combined%20widgets.ipynb

# directory containing gene expression data and network folder
directory = "."
# folder containing networks
netPath = 'Networks'
# name of gene expression file
expressionFile = 'Th0_Th17_48hTh.txt'
# sample condition for initial gene node color
sampleConditionOfInt = 'Th17(48h)' 

# The starting conditions for each of the networks is a list of tuples.  Tuple entries are:
# 0. network file name (column format) (as found in directory)
# 1. column of the expression matrix that you want the nodes to be colored by
# 2. network title, to which we'll add the gene and peak cutoffs
# 3. cut off for edge strength, note TRN edges strengths are quantile for 15 TFs/gene, to see top 10 TFs/gene, 
#    increase cutoff to .33, etc.
networkInits = [
    ('ChIP_A17_KOall_ATh_bias50_maxComb_sp.tsv',sampleConditionOfInt,' Final ChIP/ATAC(Th17)+KO+ATAC(Th) TRN',.93),
    ('ATAC_Th17_bias50_maxComb_sp.tsv',sampleConditionOfInt,'Final ATAC-only TRN', .93),
    ("KO75_KOrk_1norm_sp.tsv",sampleConditionOfInt,'KO G.S. (25 TFs)',0),
    ("KC1p5_sp.tsv",sampleConditionOfInt,'KO-ChIP G.S. (9 TFs)',0)]

tfFocus = 1 # If 1, automatically applies the "TF only" function, so we can focus on TFs
    # If 0, all genes shown

In [2]:
# Uncomment to run without install (in binder, for example)
import sys
if ".." not in sys.path:
    sys.path.append("..")
from jp_gene_viz import dNetwork
dNetwork.load_javascript_support()
from jp_gene_viz import multiple_network
from jp_gene_viz import LExpression
LExpression.load_javascript_support()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
networkList = list()  # this list will contain heatmap-linked network objects
for networkInit in networkInits:
    networkFile = networkInit[0]
    curr = LExpression.LinkedExpressionNetwork()   
    print directory + '/' + networkFile
    curr.load_network(directory + '/' + netPath + '/' + networkFile)        
    networkList.append(curr)     

./ChIP_A17_KOall_ATh_bias50_maxComb_sp.tsv
('Reading network', './Networks/ChIP_A17_KOall_ATh_bias50_maxComb_sp.tsv')
Computing layout
Using fast layout because the network is large.
('Saving layout', './Networks/ChIP_A17_KOall_ATh_bias50_maxComb_sp.tsv.layout.json')
Omitting edges, using canvas, and fast layout default because the network is large
./ATAC_Th17_bias50_maxComb_sp.tsv
('Reading network', './Networks/ATAC_Th17_bias50_maxComb_sp.tsv')
Computing layout
Using fast layout because the network is large.
('Saving layout', './Networks/ATAC_Th17_bias50_maxComb_sp.tsv.layout.json')
Omitting edges, using canvas, and fast layout default because the network is large
./KO75_KOrk_1norm_sp.tsv
('Reading network', './Networks/KO75_KOrk_1norm_sp.tsv')
Computing layout
Using fast layout because the network is large.
('Saving layout', './Networks/KO75_KOrk_1norm_sp.tsv.layout.json')
Omitting edges, using canvas, and fast layout default because the network is large
./KC1p5_sp.tsv
('Reading net

In [4]:
# visualize the networks -- HARD CODED for 4 networks:
M = multiple_network.MultipleNetworks(
    [[networkList[0], networkList[1]],
    [networkList[2], networkList[3]]])
M.svg_width = 500
M.show()  

A Jupyter Widget

In [5]:
# Set network preferences
count = 0
for curr in networkList:
    networkInit = networkInits[count]
    # get title information + curr column for shading of figures
    currCol = networkInit[1]
    titleInf = networkInit[2]
    threshhold = networkInit[3]

    # set threshold
    curr.network.threshhold_slider.value = threshhold
    curr.network.apply_click(None)
    curr.network.restore_click(None)
    if tfFocus:
        # focus on TF core    
        curr.network.tf_only_click(None)
        curr.network.layout_click(None)  
    # layout network
    curr.network.connected_only_click()
    curr.network.layout_dropdown.value = 'fruchterman_reingold'
    curr.network.layout_click()
    
    # set title
    curr.network.title_html.value = titleInf

    # add labels
    curr.network.labels_button.value=True  
    curr.network.draw_click(None)

    # Load heatmap
    curr.load_heatmap(directory + '/' + expressionFile)
    # color nodes according to a sample column in the gene expression matrix
    curr.gene_click(None)       
    curr.expression.transform_dropdown.value = 'Z score'
    curr.expression.apply_transform()
    curr.expression.col = currCol    
    curr.condition_click(None)    
    count += 1